# Randomized Dependence Coefficient

## Imports

In [1]:
import numpy as np
from scipy.stats import rankdata
from numpy.random import rand

Computes the Randomized Dependence Coefficient  
    x,y: numpy arrays 1-D or 2-D  
         If 1-D, size (samples,)  
         If 2-D, size (samples, variables)  
    f:   function to use for random projection  
    k:   number of random projections to use  
    s:   scale parameter  
    n:   number of times to compute the RDC andreturn the median (for stability)  
    According to the paper, the coefficient should be relatively insensitive to the settings of the f, k, and s parameters.

In [2]:
def rdc(x, y, f=np.sin, k=20, s=1/6., n=1):
    if n > 1:
        values = []
        for i in range(n):
            try: values.append(rdc(x, y, f, k, s, 1))
            except np.linalg.linalg.LinAlgError: pass
        return np.median(values)

    if len(x.shape) == 1: x = x.reshape((-1, 1))
    if len(y.shape) == 1: y = y.reshape((-1, 1))

    # Copula Transformation
    cx = np.column_stack([rankdata(xc, method='ordinal') for xc in x.T])/float(x.size)
    cy = np.column_stack([rankdata(yc, method='ordinal') for yc in y.T])/float(y.size)

    # Add a vector of ones so that w.x + b is just a dot product
    O = np.ones(cx.shape[0])
    X = np.column_stack([cx, O])
    Y = np.column_stack([cy, O])

    # Random linear projections
    Rx = (s/X.shape[1]) * np.random.randn(X.shape[1], k)
    Ry = (s/Y.shape[1]) * np.random.randn(Y.shape[1], k)
    X = np.dot(X, Rx)
    Y = np.dot(Y, Ry)

    # Apply non-linear function to random projections
    fX = f(X)
    fY = f(Y)

    # Compute full covariance matrix
    C = np.cov(np.hstack([fX, fY]).T)

    # Due to numerical issues, if k is too large,
    # then rank(fX) < k or rank(fY) < k, so we need
    # to find the largest k such that the eigenvalues
    # (canonical correlations) are real-valued
    k0 = k
    lb = 1
    ub = k
    while True:
       # Compute canonical correlations
        k = int(k)
        k0 = int(k0)
        Cxx = C[:k, :k]
        Cyy = C[k0:k0+k, k0:k0+k]
        Cxy = C[:k, k0:k0+k]
        Cyx = C[k0:k0+k, :k]

        eigs = np.linalg.eigvals(np.dot(np.dot(np.linalg.inv(Cxx), Cxy),
                                        np.dot(np.linalg.inv(Cyy), Cyx)))

        # Binary search if k is too large
        if not (np.all(np.isreal(eigs)) and
                0 <= np.min(eigs) and
                np.max(eigs) <= 1):
            ub -= 1
            k = (ub + lb) / 2
            continue
        if lb == ub: break
        lb = k
        if ub == lb + 1:
            k = ub
        else:
            k = (ub + lb) / 2

    return np.sqrt(np.max(eigs))

## Testing

In [3]:
a = np.array([1,2,3,4,5, 28, 3])
b = np.array([5,6,3,8,7, 44, 12])

rdc(a, b)

LinAlgError: Singular matrix

## Credits & Links

http://papers.nips.cc/paper/5138-the-randomized-dependence-coefficient.pdf